<a href="https://colab.research.google.com/github/Pavadareni/AI_Training/blob/main/Question_and_Answering/question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers faiss-cpu sentence-transformers

In [7]:
import faiss
from sentence_transformers import SentenceTransformer

retriever_model = SentenceTransformer("all-MiniLM-L6-v2")
with open("knowledge.txt", "r") as file:
    knowledge_base = file.read().splitlines()
kb_embeddings = retriever_model.encode(knowledge_base)
index = faiss.IndexFlatL2(kb_embeddings.shape[1])
index.add(kb_embeddings)

def retrieve_context(question, k=1):
    """
    Retrieve the most relevant context from the knowledge base for the given question.
    """
    question_embedding = retriever_model.encode([question])
    distances, indices = index.search(question_embedding, k)
    return [knowledge_base[idx] for idx in indices[0]]


In [8]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

def answer_question(question, context):
    """
    Use BERT to extract an answer from the given context.
    """
    result = qa_pipeline(question=question, context=context)
    return result["answer"]


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
def qa_system(question):
    """
    A QA system that retrieves the context and extracts the answer.
    """
    # Step 1: Retrieve the most relevant context
    retrieved_contexts = retrieve_context(question, k=1)
    if not retrieved_contexts:
        return "No relevant context found."

    # Step 2: Use the reader (BERT) to find the answer
    context = retrieved_contexts[0]
    answer = answer_question(question, context)
    return answer


In [10]:
if __name__ == "__main__":
    print("QA System: Ask me about IITs or related information.")
    while True:
        question = input("You: ")
        if question.lower() in ["exit", "quit"]:
            print("QA System: Goodbye!")
            break
        answer = qa_system(question)
        print(f"QA System: {answer}")
''

QA System: Ask me about IITs or related information.
You:  What is the fee for undergraduate programs at IIT Bombay?
QA System: ₹10,00,000
You: What is IIT Delhi known for?
QA System: Civil Engineering, Chemical Engineering, and Computer Science
You: is placements are avalaible
QA System: ₹16 LPA.
You: hightest package
QA System: average placement package at IIT Bombay is ₹15 LPA
You: exit
QA System: Goodbye!
